## Dual numbers and automatic differentiation

Implemented the forward mode of automatic differentiation with the help of dual numbers. We first implement a class **Dual** with the constructor **__init__**, the functions **__add__**, **__radd__**, **__sub__**, **__rsub__**, **__mul__**, **__rmul__**, **__truediv__**, **__rtruediv__**, **__neg__** and **__pow__**, and the property **T**. As the names suggest, those  functions and properties implement basic arithmetic operations for Dual numbers:

__init__ : constructor that initialises an object of class **Dual**. Each object represents a dual number $a + \varepsilon \, b$ with real component $a$ (*self.real*) and dual component $b$ (*self.dual*).

__add__ : adds an argument _argument_ to the dual number, i.e. $a + \varepsilon \, b + \text{argument}$. 

__radd__ : adds the dual number to the argument _argument_, i.e. $\text{argument} + a + \varepsilon \, b$.

__sub__ : subtracts an argument _argument_ from the dual number. 

__rsub__ : subtracts the dual number from the argument _argument_.

__mul__ : multiplies the dual number with the argument _argument_.

__rmul__ : multiplies an argument _argument_ with the dual number. 

__truediv__ : divides the dual number by an argument _argument_.

__rtruediv__ : divides the argument _argument_ by the dual number.

__neg__ : returns the neagtive of the dual number $a + \varepsilon b$, i.e. $-a - \varepsilon b$.

__pow__ : takes the _power_-th power of the dual number.


In [ ]:
import numpy as np

In [ ]:
class Dual:
    
    def __init__(self, real, dual):
        '''
        real: real number
        dual: dict (key=name_index and value=value)
        '''
        self.real = real
        self.dual = dual
        
    def __add__(self, argument):
        if isinstance(argument, Dual):
            real = self.real + argument.real
            dual = {}
            for key in self.dual:
                dual[key] = self.dual[key]
            for key in argument.dual:
                if key in dual:
                    dual[key] += argument.dual[key]
                else:
                    dual[key] = argument.dual[key]    
            return Dual(real, dual)
        else:
            return Dual(self.real + argument, self.dual)
        
    __radd__ = __add__
    
    def __sub__(self, argument):
        if isinstance(argument, Dual):
            real = self.real - argument.real
            dual = {}
            for key in self.dual:
                dual[key] = self.dual[key]
            for key in argument.dual:
                if key in dual:
                    dual[key] -= argument.dual[key]
                else:
                    dual[key] = -argument.dual[key]    
            return Dual(real, dual)
        else:
            return Dual(self.real - argument, self.dual)
        
    def __rsub__(self, argument):
        if isinstance(argument, Dual):
            real = -self.real + argument.real
            dual = {}
            for key in argument.dual:
                dual[key] = argument.dual[key]
            for key in self.dual:
                if key in dual:
                    dual[key] -= self.dual[key]
                else:
                    dual[key] = -self.dual[key]    
            return Dual(real, dual)
        else:
            return Dual(-self.real + argument, self.dual)
    
    def __mul__(self, argument):
        if isinstance(argument, Dual):
            real = self.real * argument.real
            dual = {}
            for key in self.dual:
                dual[key] = self.dual[key] * argument.real
            for key in argument.dual:
                if key in dual:
                    dual[key] += argument.dual[key] * self.real
                else:
                    dual[key] = argument.dual[key] * self.real
            return Dual(real, dual)
        else:
            dual = {}
            for key in self.dual:
                dual[key] = self.dual[key] * argument
            return Dual(self.real * argument, dual)
        
    __rmul__ = __mul__
  
    def __truediv__(self,argument):
        if isinstance(argument, Dual):
            x = argument.real
            new_arg = self.div_neg(argument)
            num = Dual(self.real, self.dual)
            num_modified = num*new_arg
            dual = {}
            for key in num_modified.dual:
                dual[key] = num_modified.dual[key] / (x*x)
            return Dual(num_modified.real / (x*x), dual)
        else:
            dual = {}
            for key in self.dual:
                dual[key] = self.dual[key] / argument
            return Dual(self.real / argument, dual)

    def __rtruediv__(self,argument):
        x = self.real
        den = Dual(self.real, self.dual)
        new_arg = self.div_neg(den)
        num_modified = argument*new_arg
        dual = {}
        for key in num_modified.dual:
            dual[key] = num_modified.dual[key] / (x*x)
        return Dual(num_modified.real / (x*x), dual)
        
    def __pow__(self, power):
        a = self.real
        dual = {}
        for key in self.dual:
            dual[key] = power*self.dual[key]*(a**(power-1))
        return Dual(a**power,dual)
    
    def __neg__(self):
        dual = {}
        for key in self.dual:
            dual[key] = self.dual[key]*(-1)
        return Dual(-self.real,dual)

    def div_neg(self, argument):
        dual = {}
        for key in argument.dual:
            dual[key] = argument.dual[key]*(-1)
        return Dual(argument.real,dual)
    
    def __str__(self):
        s = 'f = ' + str(round(self.real,6)) + '\n'
        for key in self.dual:
            s += 'f' + key + ' = ' + str(round(self.dual[key],6)) + '\n'
        return s

Next, we implement the following functions that are acting on dual numbers of the form $a + \varepsilon \, b$:
    
**log** : $\log(a + \varepsilon \, b)$

**exp** : $\exp(a + \varepsilon \, b)$

**sin** : $\sin(a + \varepsilon \, b)$

**cos** : $\cos(a + \varepsilon \, b)$

**sigmoid** : $\frac{1}{1 + \exp(-(a + \varepsilon \, b))}$

In [ ]:
def log_d(dual_number):
    dual = {}
    a = dual_number.real
    sa = np.log(a)
    for key in dual_number.dual:
        dual[key] = dual_number.dual[key]/a
    return Dual(sa, dual)

def exp_d(dual_number):
    dual = {}
    a = dual_number.real
    sa = np.exp(a)
    for key in dual_number.dual:
        dual[key] = dual_number.dual[key]*sa
    return Dual(sa, dual)

def sin_d(dual_number):
    dual = {}
    a = dual_number.real
    sa = np.sin(a)
    for key in dual_number.dual:
        dual[key] = dual_number.dual[key]*np.cos(a)
    return Dual(sa, dual)

def cos_d(dual_number):
    dual = {}
    a = dual_number.real
    sa = np.cos(a)
    for key in dual_number.dual:
        dual[key] = -np.sin(a)*dual_number.dual[key]
    return Dual(sa, dual)
    
def sigmoid_d(dual_number):
    dual = {}
    a = dual_number.real
    sa = 1 / (1 + np.exp(-a))
    for key in dual_number.dual:
        dual[key] = dual_number.dual[key]*sa*(1-sa)
    return Dual(sa, dual)

## Example-1

$$
f(x,y,z) = x^3 - 2x^2y^2 + y^3
$$
<br>
$$
f_x = \frac{\partial f}{\partial \:x} = 3x^2 - 4xy^2
$$
<br>
$$
f_y = \frac{\partial f}{\partial \:y} = 3y^2 - 4x^2y
$$

At $x=1$ and $y=2$,

$f = 1$, $f_x = -13$, $f_y = 4$

In [ ]:
x = Dual(real=1, dual={'x': 1})
y = Dual(real=2, dual={'y': 1})

f = (x**3) - 2*(x**2)*(y**2) + (y**3)
print(f)

f = 1
fx = -13
fy = 4



## Example-2

$$
f(x,y,z) = \frac{81x}{x+y^2}
$$
<br>
$$
f_x = \frac{\partial f}{\partial \:x} = \frac{81y^2}{\left(x+y^2\right)^2}
$$
<br>
$$
f_y = \frac{\partial f}{\partial \:y} = -\frac{162xy}{\left(x+y^2\right)^2}
$$

At $x=2$ and $y=4$,

$f = 9$, $f_x = 4$, $f_y = -4$

In [ ]:
x = Dual(2, {'x': 1})
y = Dual(4, {'y': 1})
f = 81*x / (x+(y**2))
print(f)

f = 9.0
fx = 4.0
fy = -4.0



## Example-3

$$
f(x,y,z) = \frac{36xz}{x+z^2+y^2}
$$
<br>
$$
f_x = \frac{\partial f}{\partial \:x} = \frac{36z\left(z^2+y^2\right)}{\left(x+z^2+y^2\right)^2}
$$
<br>
$$
f_y = \frac{\partial f}{\partial \:y} = -\frac{72xzy}{\left(x+z^2+y^2\right)^2}
$$
<br>
$$
f_z = \frac{\partial f}{\partial \:z} = \frac{36x\left(-z^2+x+y^2\right)}{\left(x+z^2+y^2\right)^2}
$$

At $x=1, y=2$ and $z=1$,

$f = 6$, $f_x = 5$, $f_y = -4$ and $f_z = 4$

In [ ]:
x = Dual(1, {'x': 1})
y = Dual(2, {'y': 1})
z = Dual(1, {'z': 1})
f = 36*x*z / (x+(z**2)+(y**2))
print(f)

f = 6.0
fx = 5.0
fz = 4.0
fy = -4.0



## Example-4

$$
f(x,y,z) = \frac{\sin(x)}{\cos(y)+x^2}
$$
<br>
$$
f_x = \frac{\partial f}{\partial \:x} = \frac{\cos \left(x\right)\left(\cos \left(y\right)+x^2\right)-2x\sin \left(x\right)}{\left(\cos \left(y\right)+x^2\right)^2}
$$
<br>
$$
f_y = \frac{\partial f}{\partial \:y} = \frac{\sin \left(x\right)\sin \left(y\right)}{\left(\cos \left(y\right)+x^2\right)^2}
$$

At $x=\pi$ and $y=\pi$,

$f = 0$, $f_y = 0$, <br><br>
$f_x = \frac{1}{1-\pi^2}= -0.112744 $

In [ ]:
x = Dual(np.pi, {'x': 1})
y = Dual(np.pi, {'y': 1})

f = sin_d(x)/(cos_d(y)+(x**2))
print(f)

f = 0.0
fx = -0.112745
fy = 0.0

